In [1]:
from llama_index.core import StorageContext, load_index_from_storage
from openai import OpenAI
from llama_index.core.postprocessor import SimilarityPostprocessor, SentenceTransformerRerank
from llama_index.embeddings.huggingface_optimum import OptimumEmbedding

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
2024-03-13 20:48:36.766392: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-13 20:48:37.424563: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-13 20:48:38.705954: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
embed_model = OptimumEmbedding("/mnt/d/bt/luan/embed_onnx/")

In [3]:
storage_context = StorageContext.from_defaults(persist_dir="/mnt/d/bt/luan/index_storage2/")

In [4]:
loaded_index = load_index_from_storage(storage_context, embed_model=embed_model)

In [5]:
retriever = loaded_index.as_retriever(similarity_top_k=5)

In [6]:
postprocessor = SentenceTransformerRerank(model='BAAI/bge-reranker-base', top_n=2)
postprocessor2 = SimilarityPostprocessor(similarity_cutoff=0.8)

In [8]:
# Point to the local server
client = OpenAI(base_url="http://192.168.1.8:1234/v1", api_key="not-needed")

system_prompt = "Bạn là một trợ lí Tiếng Việt để trả lời các câu hỏi về thuế và bảo hiểm nhiệt tình và trung thực. Hãy xem như toàn bộ các kiến thức về luật của bạn đều không thể sử dụng được. Hãy chỉ trả lời theo đúng thông tin được cung cấp, nếu không có thông tin nào được cung cấp, vui lòng không trả lời câu hỏi và đưa ra lý do, không cố gắng bịa đặt câu trả lời hoặc đưa ra câu trả lời sai sự thật.\n"
system_prompt += "Thông tin để trả lời sẽ nằm ngay sau câu hỏi người dùng, trong đó câu hỏi và thông tin cách nhau bởi 'Context:'\n"
system_prompt += "Hãy trả lời ngắn gọn, đúng trọng tâm và trích dẫn tiêu đề của ngữ cảnh được tham khảo để đưa ra câu trả lời dưới dạng 'Căn cứ xxx' với xxx là tiêu đề ngữ cảnh. "

conversation = [{"role": "system", "content": system_prompt }]
while True:
    human = input("Human: ")
    if human.lower() == "reset":
        conversation = [{"role": "system", "content": system_prompt }]
        print("The chat history has been cleared!")
        continue
    if human.lower() == "end":
        break
    retrieved_docs = retriever.retrieve(human)
    reranked_nodes = postprocessor.postprocess_nodes(retrieved_docs, query_str=human)
    filtered_nodes = postprocessor2.postprocess_nodes(reranked_nodes)
    i = 1
    query = f"Câu hỏi: {human} Context:\n"
    for doc in filtered_nodes:
        file_name = doc.metadata.get("file_name").split("/")[-1].split(".")[0]
        # print (f"Tiêu đề {i}: {file_name}:\nNội dung:\n {doc.text}\n")
        query += f"Tiêu đề {i}: {file_name}:\nNội dung:\n {doc.text}\n"
        i += 1
    conversation.append({"role": "user", "content": query })
    completion = client.chat.completions.create(
        model="local-model", # this field is currently unused
        messages=conversation,
        top_p=0.95,
        temperature=0.1,
        # stream=True
    )
    new_message = {"role": "assistant", "content": ""}
    print (completion.choices[0].message.content)
    new_message["content"] += completion.choices[0].message.content
    # for chunk in completion:
    #     if chunk.choices[0].delta.content:
    #         print(chunk.choices[0].delta.content, end="", flush=True)
    #         new_message["content"] += chunk.choices[0].delta.content
    
    print("\n\n")
    conversation.append(new_message)

APITimeoutError: Request timed out.